In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

In [19]:
#Write a function to do the same that I did with the distance_df data set
def process_distance_data(input_file, output_file):
    """
    Load a CSV file, rename a column for easier merging, and save the processed data as a new CSV file.
    
    Args:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the output CSV file.
    """
    distance_df = pd.read_csv(input_file, encoding='latin1')
    distance_df.rename(columns={'country':'mission_country'}, inplace=True)
    distance_df.to_csv(output_file, index=False)

#Apply the function to "C:\Users\HP\Desktop\Felipe\Moises\Distance\Mission_coordinatesok.csv" and save the output to "C:\Users\HP\Desktop\Felipe\Moises\Distance\epp\distance_to_missions_epp.csv"
process_distance_data("C:/Users/HP/Desktop/Felipe/Moises/Distance/Mission_coordinatesok.csv", "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distance_to_missions_epp.csv")

In [20]:
# Define the function to process shapefiles
def shape_to_csv(department_path):
    # Read and reproject the shp files
    shape = gpd.read_file(department_path)
    shape = shape.to_crs(epsg=32721)
    # Calculate the centroid and extract the longitude and latitude
    shape['centroid'] = shape.centroid
    shape['longitude'] = shape['centroid'].x
    shape['latitude'] = shape['centroid'].y
    # Save as CSV in the same folder as the shapefile for easier access
    output_csv = department_path.replace(".shp", ".csv")
    shape.to_csv(output_csv, index=False)

# Root directory containing the folders and shapefiles
root_dir = "C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files"

# Go through all subdirectories and apply the function
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".shp") and file.startswith("Distritos_"):
            department_path = os.path.join(root, file) 
            shape_to_csv(department_path)


In [21]:
def append_csvs_to_one(output_file, root_dir):
    # List to store individual DataFrames
    all_data = []

    # Walk through all subdirectories to find CSV files
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".csv"):  # Process only CSV files
                file_path = os.path.join(root, file)
                print(f"Appending: {file_path}")
                # Read the CSV and append to the list
                df = pd.read_csv(file_path)
                all_data.append(df)

    # Concatenate all DataFrames into one
    deparmentos_shape_ok = pd.concat(all_data, ignore_index=True)

    # Save the combined DataFrame as a single CSV
    deparmentos_shape_ok.to_csv(output_file, index=False)
    print(f"All CSV files appended and saved to: {output_file}")

#Apply the function
root_dir = "C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files"
output_file = "C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv"

append_csvs_to_one(output_file, root_dir)

Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\guaira_shp_data.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\alto_parana\Distritos_Alto_Parana.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\caazapa\Distritos_Caazapa.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\guaira\Distritos_Guaira.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\itapua\Distritos_Itapua.csv
Appending: C:/Users/HP/Desktop/Felipe/Moises/Distance/shape_files\misiones\Distritos_Misiones.csv
All CSV files appended and saved to: C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distritos_combined.csv


In [22]:
#Now, generate distance variable for the departments centroid to the missions
# Load the department data
#departments = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/departments_combined.csv")

# Load the distance data
#distance_to_missions = pd.read_csv("C:/Users/HP/Desktop/Felipe/Moises/Distance/epp/distance_to_missions_epp.csv")

# Merge the department and distance data
#departments_distance = departments.merge(distance_to_missions, on='mission_country')
